In [1]:
#Counterfactual Regret Minimization implementation
#Rock Paper Scissors example
import itertools
import numpy as np

NUM_ACTIONS = 3
NUM_PLAYERS = 2
ROCK, PAPER, SCISSORS = range(NUM_ACTIONS)

A = np.array(list(itertools.product(range(NUM_ACTIONS), repeat=NUM_PLAYERS))) #all combinations of simul actions

In [2]:
class Player:
    def __init__(self, regrets_sum=np.zeros(NUM_ACTIONS)):
        self._strategy = np.zeros(NUM_ACTIONS)
        self.strategy_sum = np.zeros(NUM_ACTIONS)
        self.regrets_sum = regrets_sum
        self.number_of_updates = 0
    
    @property
    def strategy(self):
        self._strategy = np.maximum(self.regrets_sum, 0)
        self._strategy = self._strategy/sum(self._strategy) if sum(self._strategy) > 0 \
                    else np.ones(NUM_ACTIONS)/NUM_ACTIONS
        self.strategy_sum += self._strategy
        return self._strategy
    
    def payoff(self, my_action, other_action):
        mod_3 = (my_action - other_action) % 3
        if mod_3 == 2:
            return -1
        else:
            return mod_3
    
    def update_regrets(self, other_action, my_payoff):
        self.regrets_sum += np.array([self.payoff(a, other_action) - my_payoff for a in range(NUM_ACTIONS)])
    
    def update_strategy(self, my_action, other_action):
        my_payoff = self.payoff(my_action, other_action)
        self.update_regrets(other_action, my_payoff)
        self.number_of_updates += 1
        

In [21]:
opp_strategy = np.array([.3, .3, .4])
me = Player(regrets_sum=np.array([400, 150, 0]))
opp = Player()
my_optimal, opp_optimal = train(100000)
my_optimal, opp_optimal


(array([0.33594638, 0.33138933, 0.33266428]),
 array([0.33816863, 0.32780438, 0.33402699]))

In [29]:
opp.strategy_sum / opp.number_of_updates
print(expectedUtility([1-.25-.6,.25,.6], mixed))
expectedUtility(mixed, [1-.25-.6,.25,.6])

0.0


0.0

In [5]:
def get_action(strategy):
    return np.random.choice(range(NUM_ACTIONS), p=strategy)

In [6]:
def train(iterations):
    for _ in range(iterations):
        #⟨Get regret-matched mixed-strategy actions⟩
        my_action = get_action(me.strategy)
        opp_action = get_action(opp.strategy)
        me.update_strategy(my_action, opp_action)
        opp.update_strategy(opp_action, my_action)
    
    my_optimal = me.strategy_sum / me.number_of_updates
    opp_optimal = opp.strategy_sum / opp.number_of_updates    
    return my_optimal, opp_optimal

In [7]:
#random things i learned while cfr
def getValue(c):
    c1, c2 = c
    if c1==c2:
        return 0
    elif c1==0 and c2==2:
        return 1
    elif c1==1 and c2==0:
        return 1
    elif c1==2 and c2==1:
        return 1
    return -1

def payoffTable(p1v, p2v):
    out = np.empty(len(p1v), dtype=object)
    out[:] = list(zip(p1v, p2v))
    return out.reshape(3,3)

def expectedUtility(pi1, pi2):
    prob_chart = np.array([pi1[a1]*pi2[a2] for a1,a2 in A])
    payoff_chart = np.array([getValue(a) for a in A])
    u = prob_chart*payoff_chart
    return sum(u)

In [8]:
mixed = np.ones(NUM_ACTIONS)/NUM_ACTIONS
p1v = [getValue(a) for a in A]
p2v = [getValue(a) for a in A]

In [9]:
pi1 = [1/4, 1/2, 1/4]
pi2 = [.5, .25, .25]
expectedUtility(pi1, pi2) == 1/16

True